# Ensemble Learning and Random Forests 

An aggregated answer is almost always better than a single answer (wisdom of the crowd). A group of predictors is called an _ensemble_ and this machine learning technique is known as _Ensemble Learning_. One such algorithm in Ensemble Learning is the _Ensemble Method_. One implementation of this would be training multiple Decision Tree Classifiers on different subsets of the training data in a group called a _Random Forest_. Simply use the predictions from each class and output the most frequent prediction. Despite it's simplicity, this is one of the most powerful machine learning algorithms available.

These are frequently used at the end of a project when a few good predictors have been built. Several machine learning competition winners use multiple Ensemble methods.

We will look at _bagging_, _boosting_, _stacking_, and some others, along with Random Forests.

## Voting Classifiers

Suppose you have multiple different classifiers each with 80% or better accuracy. We can aggregate the predictions of these classifiers and use the most popular vote as our final prediction. This is called a _hard-voting_ classifier. This aggregated classifier is oftentimes stronger than it's strongest member, and an ensemble composed of _weak learners_ (not much better than random guessing) can become a _strong learner_ under certain conditions. This is possible through the _law of large numbers_: the ratio of correct predictions grows as more diverse, independent, and differently-faulted models are making predictions. In practice this rarely happens since each model is trained on the same data.

Ensemble methods work best when predictors are as diverse as possible. This can be done through the algorithms used, such using a Decision Tree, Support Vector Classifier, a Logorithmic Classifier, etc. to ensure that each model is making errors in different places. This will ensure the accuracy of the Ensemble.

The following code trains a voting classifier on three diverse classifiers:

In [23]:
import sklearn
from sklearn.datasets import make_moons
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

X, y = make_moons(n_samples=100, noise = .25);

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf) ],
    voting='hard'
)
#voting_clf.fit(X_train, y_train);

Let's look at each classifiers accuracy: 

In [24]:
from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.8
RandomForestClassifier 0.95
SVC 0.85
VotingClassifier 0.8


If all classifiers have a predict_proba() method, then we can assign weights to each models prediction and select the highest class probability. This is also known as _soft_ voting. This usually achieves better results by assigning a stronger weight to more confident votes.

## Bagging and Pasting

Another approach to getting a diverse set of classifiers is by using the same classifier trained on different subsets of the training data. When sampling is performed _with_ replacement, this is called _bootstrap aggregating_ or _bagging_ for short. When it's performed without replacement, this is called _pasting_. Bagging and pasting allow the data to be sampled multiple times across different predictors, but only bagging allows individual instances to be sampled multiple times. 

After the predictors are trained, the final prediction can be made by aggregating the predictions from individual predictors using the _statistical mode_ for classification and the average for regression. Although each predictor has a higher bias than one trained on the entire set, the use of an ensemble will lower both bias and variance. Generally, an ensemble will have similar bias but lower variance.

Predictors in an ensemble can be trained in parallel across multiple servers or even CPU cores. Likewise, predictions can also be made in parallel. This scalability makes bagging a popular method.

## Bagging and Pasting in Scikit-learn

We can use the BaggingClassifier or BaggingRegressor in scikit-learn. The following trains 500 decision trees on 100 training instances with replacement. the n_jobs parameter indicates the number of CPU cores to use (-1 uses all available cores).

In [26]:
from sklearn.ensemble import  BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
        DecisionTreeClassifier(), n_estimators=500,
        max_samples=80, bootstrap=True, n_jobs=-1
    )
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.9


Bootstrapping will result with a slightly higher bias and lower variance since the predictors are less correlated. Use cross-validation to evaluate which works best.


## Out-of-Bag Evaluation

With bagging, some samples may be used multiple times and other not at all. On average, only 63% of training instances are used on each predictor. The remaining 37% are called _out-of-bag_ (oob) instances. Since a predictor never sees the oob instances, these can be used for evaluating that predictor without needing a seperate validation set or cross-validation. One can even evaluate the entire ensemble on these by using the average oob instances from each predictor. We can see the oob validation score in scikit-learn like so:

In [27]:
bag_clf = BaggingClassifier(
        DecisionTreeClassifier(), n_estimators=500,
        bootstrap=True, n_jobs=-1, oob_score=True)
bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

0.91249999999999998

Let's verify this score:

In [28]:
accuracy_score(y_pred, y_test)

0.90000000000000002

Close enough!

## Random Patches and Random Subspaces

The BaggingClassifier supports sampling features as well. The max_features and bootstrap_features work kist like max_samples and bootstrap but for feature sampling instead of data sampling. This is highly useful for high-dimenisional inputs. Sampling both instances and features is known as the _Random Patches_ method. Keeping all instances while sampling features known as the _Random Subspaces_ method. Feature sampling results in even higher predictor diversity, again raising bias for lower variance.

## Random Forests

Random Forests are usually trained via bagging on the entire training set (max_samples). In scikit-learn, we can use the RandomForestClassifier instead of a BaggingClassifier fed with a DecisionTreeClassifier. Not only is this more convenient; it's also better optimized. Here is a RandomForestClassifier trained with 500 trees each with no more than 16 nodes on all CPU cores:

In [30]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)
print(accuracy_score(y_pred_rf, y_test))

0.95


Nice! A RandomForestClassifier comes with nearly all the hyperparameters as a DecisionTreeClassifier and all the hyperparameters of a BaggingClassifier.

This type of classifier adds an additional degree of randomness by searching for the best feature among a subset of features, instead of searching the entire feature set. This results in greater predictor diversity which results in (again) slightly higher bias and lower variance, generally creating a better model. This bagging classifier is roughly equivalent to the above random forest:

In [31]:
bag_clf = BaggingClassifier(
        DecisionTreeClassifier(splitter="random", max_leaf_nodes=16),
        n_estimators=500, max_samples=1.0, bootstrap=True, n_jobs=-1
)

## Extra-Trees

In a Random Forest Classifier, only a random subset of the features is considered for splitting. This can be made even more random by also using random thresholds for each feature rather than searching for the best possible thresholds.

A forest of such trees is called an _Extremely Randomized Trees_ ensemble, or _Extra-Trees_ for short. This trades even more bias for lower variance. It also makes trees much faster to train than regular Random Forests since finding the best threshold is one of the most time consuming tasks in growing a tree. 

These can be utilized in scikit-learn with the ExtraTreesClassifier or ExtraTreesRegressor, which are identical to their RandomForest counterpart.

It is difficult to tell in advance if a Random Forest or Extra Trees classifier/regressor will perform better than the other. Generally, it is best to use cross-validation to compare the models and use grid-search to find the best hyperparameter values.

## Feature Importance

Looking at a single Decision tree, we can observe that more important features are likely to appear near the root node while unimportant features are found near the leaves or not at all. It is possible to get a features importance by computing the average depth at which that feature is found. Scikit-Learn does this automatically after training and can be accessed with the feature\_importances<span>\_</span> variable. Let's see what the most important features are in the iris dataset:

In [34]:
from sklearn.datasets import load_iris
iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris["data"], iris["target"])
for name, score in zip(iris["feature_names"], rnd_clf.feature_importances_):
    print(name, score)

sepal length (cm) 0.0854747740243
sepal width (cm) 0.0218246918336
petal length (cm) 0.442351990073
petal width (cm) 0.450348544069


Petal width and Petal length are the most important features in classifying a new iris. A similar method can be performed on the MNIST dataset to evaluate each pixels importance. This quality of Random Forests make them very handy for getting a quick understanding of which features actually matter and are extrememly helpful when performing feature selection.

## Boosting

_Boosting_ (originally _hypothesis boosting_) refers to any Ensemble method that can comine several weak learners into a strong learner. Generally, boosting methods will train predictors sequentially and work to correct the mistakes of it's predecessors. The two most popular are _AdaBoost_ (short for _Adaptive Boosting_) and _Gradient Boosting_.

### AdaBoost

One way to correct the predecessor is to pay more attention to the training instances that the predeccor underfitted. This results in predictors focusing more resources on the hard cases and is the technique used in AdaBoost.

To build an AdaBoost Classifier, a base classifier is trained and used to make predictions on the **training set**. The relative weight of misclassified training instances is then increased. A second classifier is then trained, makes predictions, then the weights are updated, etc. Misclassified instances have their weights "boosted" over each iteration. This technique shares some similarities with Gradient Descent, except instead of modifying a single predictors parameters to minimize a cost function, AdaBoost adds predictors to the Ensemble.

Once each predictor is trained, the ensemble makes predictions just like bagging or pasting, except that predictors have different weights depending on their overall accuracy on the weighted training set. One huge drawback of this method is scalability: this process cannot be parallelized well since each predictor can only be trained immidiately after the predecessor has completed it's training.

Let's take a look at the algorithm. Each instance weight $w^(i)$ is initially set to $\frac{1}{m}$. A first predictor is trained and it's weighted error rate $r_1$ is computed on the training set, equal to:

$$r_j = \frac{\sum_{\substack{i = 1\\ \hat{y}_j^{(i)}\neq y^{(i)} }}^{m} w^{(i)}}{\sum_{i = 1}^{m} w^{(i)}}$$

where ${\hat{y}}_j^{(i)}$ is the $j$<sup>th</sup> predictor's prediction for the $i$<sup>th</sup> instance.

The predictor's weight $\alpha_j$ is then computed like so:

$$\alpha_j = \eta \log\frac{1-r_j}{r_j}$$

where $\eta$ is the learning rate hyperparameter (defaults to 1). The more accurate the predictor, the higher it's weight will be. Random guessing results in a weight of 0 while worse-than-random guessing results in a negative weight. Next, instance weights are updated like so:

$$w^{(i)} \leftarrow \Big{ {{w^{(i)} if \hat{y}_{j}^{(i)} = y^{(i)}} \\ {w^{(i)} \exp(\alpha_{j}) if \hat{y}_{j}^{(i)} \neq y^{(i)}}}$$

Then each instance weight is normalized by dividing by the sum of the weights. Finally a new predictor is trained using the updated weights and the whole process is repeated (compute new weights, update instance weights, train another predictor). The algorithm stops when the stated number of predictors has been found or a perfect predictor is produced. 

To make predictions AdaBoost computes predictions from each predictor and weighs them with the predictor weights. The chosen class is the one th receives the highest weighted vote.

Scikit-LEarn uses a multiclass version of AdaBoost called _Stagewise Additive Modeling using a MultiClass Exponential loss function_ or _SAMME_. If the predictors can estimate class probabilties then SAMME.R ("Real") is used using class probabilites. This generally performs better than SAMME. Here's an implementation using an AdaBoostClassifier:

In [36]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
        DecisionTreeClassifier(max_depth=1), n_estimators=200,
        algorithm="SAMME.R", learning_rate=0.5
    )
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'),
          learning_rate=0.5, n_estimators=200, random_state=None)

If the AdaBoost Ensemble is overfitting the training set, you can reduce the number of estimators or more strongly regularize the base estimator.

### Gradient Boosting

This method also works by sequentially training predictors and adding them to the ensemble. However, instead of tweaking instance weights this method tries to fit the new predictor using the _residual errors_ made by the previous predictor.

Gradient Boosting is well-suited to regression tasks. In this example we'll use a Decision Tree Regressor as the base predictor in what is called _Gradient Tree Boosting_ or _Gradient Boosted Regression Trees_.

In [42]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor

X = np.random.rand(100, 1)
X = X[:] - .5
y = X*X + np.random.rand(100,1)

tree_reg1 = DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X, y)

DecisionTreeRegressor(criterion='mse', max_depth=2, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best')

Now, we'll train a second Decision tree on the residual errors made by the first predictor:

In [44]:
y2 = y - tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X, y2)

DecisionTreeRegressor(criterion='mse', max_depth=2, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best')

We'll train a third predictor on the residual errors of the second predictor:

In [45]:
y3 = y2 - tree_reg2.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X, y3)

DecisionTreeRegressor(criterion='mse', max_depth=2, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best')

Now we have an ensemble of three trees and can make predictions on a new instance by simply adding up the predictions of each tree.

In [49]:
X_new = .3
y_pred = sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))
print(y_pred)

[[ 0.53479784  0.53479784  0.53479784 -0.11580162  0.3940975   0.53479784
   0.53479784  0.53479784  0.53479784  0.53479784  0.53479784  0.53479784
   0.3940975   0.53479784  0.53479784  0.53479784  0.53479784  0.53479784
   0.53479784  0.53479784  0.53479784  0.53479784  0.3940975   0.53479784
   0.3940975   0.3940975   0.53479784  0.53479784  0.53479784  0.3940975
   0.53479784  0.3940975   0.53479784  0.53479784  0.3940975   0.53479784
   0.53479784  0.3940975   0.3940975   0.53479784  0.09537083  0.53479784
   0.53479784  0.3940975   0.53479784  0.53479784  0.53479784  0.53479784
   0.53479784  0.53479784  0.53479784  0.3940975   0.53479784  0.3940975
   0.3940975   0.53479784  0.3940975   0.53479784 -0.11580162  0.53479784
   0.53479784  0.53479784  0.53479784  0.53479784  0.3940975   0.3940975
   0.53479784  0.53479784  0.53479784  0.3940975   0.53479784  0.3940975
   0.3940975   0.3940975   0.3940975   0.53479784  0.53479784  0.3940975
   0.53479784  0.53479784  0.53479784  0.39

A similar way to train a GBRT is using the GradientBoostingRegressor class. It contains hyperparameters to control the growth of trees as well as hyperparameters to control ensemble training. Here's an example:

In [56]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0)
#y = y.reshape(100, ) #Suppresses an error generated by the .fit method
gbrt.fit(X, y)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=1.0, loss='ls', max_depth=2, max_features=None,
             max_leaf_nodes=None, min_impurity_split=1e-07,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=3, presort='auto',
             random_state=None, subsample=1.0, verbose=0, warm_start=False)

The learning_rate hyperparameter scales the individual contribution of each tree. a low value will require more trees to fit the training set but the predictions will generalize better. This technique is called _shrinkage_. Beware, too many trees will overfit the data.

In order to find the optimal number of trees, we can use early stopping (see chapter 4). This can be implemented with the staged_predict() method. This returns an iterator over the predictions made by the ensemble at each stage of training (i.e. each tree). We'll train an ensemble of 120 trees and then measure the validation error at each stage to find the optimal number of trees:

In [61]:
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val = train_test_split(X, y)

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120)
gbrt.fit(X_train, y_train)
#y_pred_gbrt = gbrt.predict(X_val)
#print(accuracy_score(y_val, y_pred_gbrt))

errors =[mean_squared_error(y_val, y_pred)
        for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators = np.argmin(errors)

gbrt_best = GradientBoostingRegressor(max_depth=2, n_estimators=bst_n_estimators)
gbrt_best.fit(X_train, y_train)
print(bst_n_estimators)
#y_pred_gbrt = gbrt_best.predict(X_val)
#print(accuracy_score(y_val, y_pred_gbrt))

53


We can implement early stopping by setting warm_start=True which makes scikit-learn keep existing trees when the .fit() isn called. The following code will stop training when the validation error does not improve for five consecutive iterations:

In [62]:
gbrt = GradientBoostingRegressor(max_depth=2, warm_start=True)

min_val_error = float("inf")
error_going_up = 0
for n_estimators in range(1, 120):
    gbrt.n_estimators = n_estimators
    gbrt.fit(X_train, y_train)
    y_pred = gbrt.predict(X_val)
    val_error = mean_squared_error(y_val, y_pred)
    if val_error < min_val_error:
        min_val_error = val_error
        error_going_up = 0
    else:
        error_going_up += 1
        if error_going_up == 5:
            print(n_estimators)
            break #early stopping

59


The GradientBoostingRegressor also supports a subsample hyperparameter which specifies the fraction of training instances to train each tree, selected randomly. This as usual trades more bias for less variance while also considerably speeding training time. This technique is called _Stochastic Gradient Boosting_.

Gradient Boosting can be done with other cost functions. In scikit-learn this is done with the loss hyperparameter.

## Stacking

_Stacking_, short for _stacked generalization_, is based on training a model to perform the aggregation of predictions, instead of using trivial like those previously examined. Here, a final predictor called a _blender_ or _meta learner_ takes the predictions as inputs and makes the final prediction. A common approach to training the blender is to use a hold-out set. In practice, the training set is split into two subsets. The first u\is used to train the first-layer predictors. Next, the first-layer predictors make predictions based on the second subset. This ensure "clean" predictions. For each instance in the hold-out set there are $n$ predicted values for each predictor, making a new $n$ dimensional training set. We can create this new training set using these predicted values as input features and keeping their targets. The blender is trained on this new set so it learns predict the target based on the first layers predictions.

Several blenders can be trained this way, such as one using Linear Regression, one with Random Forest Regression, etc. creating a whole layer of blenders. To do this, the training set would need to be split into 3 subsets: the first to train the Layer 1 predictors, the second to create the training set for the Layer 2 blenders, and third to create the training set for the layer 3 super-blender with the Layer 2 blenders. Once this is complete, new instances can be predicted by going through each layer.